In [89]:
from modules.utils import *
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

### 파일 분리

In [3]:
filepath = "./test/seq2seq.pdf"

file_paths = split_pdf(filepath)

총 페이지 수: 9
./test/seq2seq
분할 PDF 생성: ./test/seq2seq_0000_0004.pdf
./test/seq2seq
분할 PDF 생성: ./test/seq2seq_0005_0008.pdf


### upstage에서 데이터 파싱

In [4]:
analysis_file_path = upstage_layout_analysis(file_paths)

In [7]:
# analysis_file_path = ['./test/hana_0000_0004.json', './test/hana_0005_0008.json']
analysis_file_path

['./test/seq2seq_0000_0004.json', './test/seq2seq_0005_0008.json']

### 각 파일 분리하여 저장
- page 번호 조정
- id 조정
- html id tag 조정

In [ ]:
element_content = []
output_file_basename = os.path.splitext(filepath)[0]
print(output_file_basename)
page_range = 5
last_number = 0
for i, path in enumerate(sorted(analysis_file_path)):

    with open(path, "r", encoding="utf-8") as file:
        # 파일 내용을 파이썬 객체로 변환
        data = json.load(file)  # 보통 JSON 배열이면 list 타입이 됨
        html_str = data["content"]["html"]

        change_page = page_range * i
        print(last_number)
        for j, element in enumerate(data["elements"]):
            element["id"] = last_number + j
            html_content = element["content"]["html"]
            soup = BeautifulSoup(html_content, "html.parser")
            tag = soup.find(attrs={"id": True})
            tag["id"] = last_number + j
            # for tag in soup.find_all(attrs={"id": True}):
            # # 여기서 id 속성을 원하는 값으로 변경
            #     tag['id'] = last_number+j

            element["content"]["html"] = str(tag)
            element["page"] = change_page + element["page"]
        last_number = len(data["elements"])
        element_content.extend(data["elements"])


with open(f"{output_file_basename}_merged.json", "w", encoding="utf-8") as json_file:
    json.dump(
        element_content,
        json_file,
    )

./test/seq2seq
0
60


In [9]:
output_file_basename

'./test/seq2seq'

### 분리한 파일에서 이미지 가져와서 저장

In [ ]:
import json
import base64
import os
from bs4 import BeautifulSoup

input_json_path = f"{output_file_basename}_merged.json"  # JSON 파일 경로
output_dir = output_file_basename  # 이미지 저장 폴더
print(input_json_path)
# 저장 폴더가 없으면 생성
os.makedirs(output_dir, exist_ok=True)
with open(input_json_path, "r", encoding="utf-8") as f:
    data = json.load(f)  # JSON 배열 데이터 로드

arr = []
for idx, item in enumerate(data):
    _dict = dict()
    if "base64_encoding" in item:
        _dict = {"base64_encoding": item["base64_encoding"]}
        html_str = item["content"]["html"]
        soup = BeautifulSoup(html_str, "html.parser")
        img_tags = soup.find_all("img")
        if img_tags:
            base64_str = item["base64_encoding"]
            # base64 문자열 디코딩
            image_data = base64.b64decode(base64_str)
            image_path = os.path.join(f"{output_dir}/image_{idx}.png")
            for img_tag in soup.find_all("img"):
                # 기존 속성 모두 제거
                img_tag.attrs.clear()
                # 원하는 텍스트를 src 속성에 넣기
                img_tag["src"] = image_path
            # 이미지 파일 저장
            _dict["content_text"] = str(img_tag)
            with open(f"{image_path}", "wb") as img_file:
                img_file.write(image_data)
            print(f"Saved image: {image_path}")
        else:
            _dict["content_text"] = item["content"]["html"]
            print("이미지 태그가 없습니다.")
    else:
        _dict["content_text"] = item["content"]["html"]
        print("base64_encoding params 없습니다.")

    _dict["metadata"] = {"id": item["id"], "page": item["page"]}
    arr.append(_dict)

with open(f"{output_file_basename}_merged_1.json", "w", encoding="utf-8") as json_file:
    json.dump(
        arr,
        json_file,
    )

./test/seq2seq_merged.json
base64_encoding params 없습니다.
이미지 태그가 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
Saved image: ./test/seq2seq/image_12.png
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding par

### content_text 에서 html 태그 가져와 tag에 img 존재하는 경우 모아서 summary 진행 

In [ ]:
import json
import base64
import os
from bs4 import BeautifulSoup


def get_system_prompt():
    return """You are an expert in extracting useful information from IMAGE.
    In particular, you specialize in analyzing papers.
    With a given image, your task is to extract key entities, summarize them, and write useful information that can be used later for retrieval."""


def get_user_image_template(previous_context, next_context, image_paths, id):
    return f"""
        Please consider the following text context—both the preceding paragraph and the following paragraph—along with the included image. 
        Based on this context, provide a simple and clear explanation of the image, suitable for someone with no prior background knowledge. 
        Use easy-to-understand language and focus on the main ideas that the image conveys in relation to the text.

        [Preceding paragraph text]
        {previous_context}
        [Following paragraph text]
        {next_context}


        ###
        Output Format:
        <image>
        <title>
        <summary>
        <entities> 
        <path> {image_paths} </path>
        <id> {id} </id>
        </image>
"""


image_urls: list[str] = []
system_prompts: list[str] = []
user_prompts: list[str] = []
# 저장 폴더가 없으면 생성
input_json_path = f"{output_file_basename}_merged_1.json"  # JSON 파일 경로
print(input_json_path)
with open(input_json_path, "r", encoding="utf-8") as f:
    data = json.load(f)  # JSON 배열 데이터 로드

for idx, item in enumerate(data):
    previous_context = data[idx - 1]["content_text"] if idx - 1 >= 0 else None
    next_context = data[idx + 1]["content_text"] if idx + 1 < len(data) else None
    id_str = item["metadata"]["id"]
    html_str = item["content_text"]
    soup = BeautifulSoup(html_str, "html.parser")
    img_tag = soup.find("img")
    # equation_tag = soup.find(attrs={'data-category': 'equation'})
    if img_tag:
        src_path = img_tag.get("src")
        image_urls.append(src_path)
        system_prompts.append(get_system_prompt())
        user_prompts.append(
            get_user_image_template(previous_context, next_context, html_str, id_str)
        )


# print(image_urls)
for u in user_prompts:
    print(u + "\n")

./test/seq2seq_merged_1.json

        Please consider the following text context—both the preceding paragraph and the following paragraph—along with the included image. 
        Based on this context, provide a simple and clear explanation of the image, suitable for someone with no prior background knowledge. 
        Use easy-to-understand language and focus on the main ideas that the image conveys in relation to the text.

        [Preceding paragraph text]
        <p data-category="paragraph" id="11" style="font-size:20px">There have been a number of related attempts to address the general sequence to sequence learning<br/>problem with neural networks. Our approach is closely related to Kalchbrenner and Blunsom [18]<br/>who were the first to map the entire input sentence to vector, and is related to Cho et al. [5] although<br/>the latter was used only for rescoring hypotheses produced by a phrase-based system. Graves [10]<br/>introduced a novel differentiable attention mechanism tha

In [ ]:
from langchain_teddynote.models import MultiModal

llm = get_gpt()
multimodal_llm = MultiModal(llm)
answer = multimodal_llm.batch(
    image_urls, system_prompts, user_prompts, display_image=False
)
print(answer)

['<image>\n<title>Sequence-to-Sequence Model Processing an Input Sentence</title>\n<summary>\nThis image shows how a special type of neural network called an LSTM processes an input sentence and generates an output sentence. The input sentence consists of the letters "A", "B", and "C". The model reads these letters one by one and then produces an output sequence "W", "X", "Y", "Z", followed by a special end-of-sentence token ("<EOS>"). The arrows indicate the flow of information through the network. The model stops generating output once it produces the "<EOS>" token. Importantly, the model reads the input sentence in reverse order to make learning easier.\n</summary>\n<entities>\n- LSTM (Long Short-Term Memory) neural network\n- Input sentence: "A", "B", "C"\n- Output sentence: "W", "X", "Y", "Z", "<EOS>"\n- End-of-sentence token ("<EOS>")\n- Sequence-to-sequence learning\n</entities>\n<path> <img src="./test/seq2seq/image_12.png"/> </path>\n<id> 12 </id>\n</image>', '<image>\n<title>

In [ ]:
# arr = ['<image>\n<title>도표 8. DDR4 가격 추이</title>\n<summary>이 그래프는 2021년 1월부터 2025년 8월까지 DDR4 메모리 가격의 변동 추이를 보여준다. 그래프에는 현물 가격과 고정 가격 두 가지가 표시되어 있으며, 초기에는 가격이 약 5달러에서 시작해 점차 하락하여 2023년 중반에는 1.5달러 근처까지 떨어졌다. 이후 가격은 다시 상승세로 돌아서 2025년 8월에는 약 6달러에 도달하는 모습을 나타낸다.</summary>\n<entities>\n- DDR4 메모리\n- 현물 가격\n- 고정 가격\n- 가격 변동 추이\n- 기간: 2021년 1월 ~ 2025년 8월\n- 단위: 달러 ($)\n</entities>\n<path> <img src="./test/hana/image_43.png"/> </path>\n<id> 43 </id>\n</image>', '<image>\n<title>도표 9. DDR5 가격 추이</title>\n<summary>이 그래프는 2023년 1월부터 7월까지 DDR5 메모리 가격의 변동 추이를 보여준다. 두 가지 가격이 표시되어 있는데, 하나는 현물 가격(진한 녹색 선)이고 다른 하나는 고정 가격(회색 선)이다. 초기에는 가격이 약 5.5달러에서 시작해 3.8달러까지 하락했다가 이후 점차 상승하여 7달러 이상으로 증가하는 추세를 보인다. 고정 가격은 현물 가격보다 낮거나 비슷한 수준에서 움직이다가 후반부에 상승하는 경향을 나타낸다.</summary>\n<entities>DDR5, 가격 추이, 현물 가격, 고정 가격, 2023년 1월~7월, 달러($)</entities>\n<path> <img src="./test/hana/image_46.png"/> </path>\n<id>46</id>\n</image>', '<image>\n<title>도표 11. SLC 8Gb 가격 추이 (SLC 8Gb Price Trend)</title>\n<summary>\nThis line graph shows the price trend of SLC 8Gb memory chips over time, from January 2021 (21.1) to August 2025 (25.8). The vertical axis represents the price in dollars ($), ranging from 0 to 12. Two price lines are depicted: the 현물 가격 (spot price) in blue-green and the 고정 가격 (fixed price) in gray. The spot price starts around $5.5 in early 2021, rises to above $8 by early 2022, then declines and stabilizes around $6.5 until early 2024. After that, it sharply increases to nearly $10 by mid-2025 before slightly dropping. The fixed price remains relatively stable around $4.5 until late 2024, then drops sharply to below $2.5 before gradually rising again to about $3.5 by mid-2025.\n</summary>\n<entities>\n- SLC 8Gb memory chip\n- Price in USD ($)\n- 현물 가격 (Spot Price)\n- 고정 가격 (Fixed Price)\n- Time period: January 2021 to August 2025\n- Data sources: DRAMExchange, 하나증권 (Hana Securities)\n</entities>\n<path> <img src="./test/hana/image_50.png"/> </path>\n<id> 50 </id>\n</image>', '<image>\n<title>현물 가격과 고정 가격 추이 (2021년 1월 ~ 2025년 8월 예상)</title>\n<summary>이 그래프는 2021년 1월부터 2025년 8월까지의 현물 가격과 고정 가격 변동 추이를 보여준다. 현물 가격은 2022년 초에 약 4달러까지 상승했다가 이후 점차 하락하여 2024년 초까지 2.7달러 근처에서 유지되다가 2025년부터 다시 상승하는 경향을 보인다. 고정 가격은 현물 가격보다 전반적으로 낮거나 비슷한 수준을 유지하며, 2024년 중반 이후부터는 현물 가격보다 낮은 수준에서 완만하게 상승하는 모습을 나타낸다.</summary>\n<entities>\n- 현물 가격 (Spot Price)\n- 고정 가격 (Fixed Price)\n- 기간: 2021년 1월 ~ 2025년 8월 (예상 포함)\n- 단위: 달러($)\n- 출처: DRAMExchange, 하나증권\n</entities>\n<path> <img src="./test/hana/image_52.png"/> </path>\n<id> 52 </id>\n</image>', '<image>\n<title>도표 12. 주요 DRAM 업체들의 연초 이후 주가 추이</title>\n<summary>이 그래프는 2024년 12월 31일을 기준으로 주요 DRAM 업체들인 삼성전자, SK하이닉스, Micron, Nanya의 주가 변동 추이를 나타내고 있다. 각 업체의 주가는 2024년 1월부터 9월까지의 기간 동안 변동을 보였으며, 특히 Nanya의 주가가 가장 큰 폭으로 상승하여 210 이상을 기록한 반면, Micron은 상대적으로 안정적인 흐름을 보이며 100 근처에서 움직였다. 삼성전자와 SK하이닉스는 중간 수준의 상승세를 유지하였다.</summary>\n<entities>\n- 삼성전자 (Samsung Electronics)\n- SK하이닉스 (SK Hynix)\n- Micron\n- Nanya\n- 주가 추이 (Stock Price Trend)\n- 기간: 2024년 1월 ~ 9월\n</entities>\n<path> <img src="./test/hana/image_57.png"/> </path>\n<id> 57 </id>\n</image>']

# html_str = data[0]  # 배열에서 첫번째 문자열 추출

input_json_path = f"{output_file_basename}_merged_1.json"  # JSON 파일 경로
with open(input_json_path, "r", encoding="utf-8") as f:
    data = json.load(f)  # JSON 배열 데이터 로드

for html_str in answer:
    soup = BeautifulSoup(html_str, "html.parser")
    id_tag = soup.find("id")  # id 태그 찾기
    if id_tag:
        id_value = id_tag.text.strip()  # 텍스트 추출 후 공백 제거
        # data[id_value]['content_text'] = html_str
        data[int(id_value)]["content_text"] = html_str
        print("id 값:", id_value)
    else:
        print("id 태그를 찾을 수 없습니다.")

with open(f"{output_file_basename}_merged_2.json", "w", encoding="utf-8") as json_file:
    json.dump(
        data,
        json_file,
    )

id 값: 12
id 값: 68
id 값: 69
id 값: 75


### 방정식 설명 추가 로직

In [52]:
import json
import base64
import os
from bs4 import BeautifulSoup


def get_system_prompt():
    return """You are an expert in extracting useful information from IMAGE.
    In particular, you specialize in analyzing papers.
    With a given image, your task is to extract key entities, summarize them, and write useful information that can be used later for retrieval."""


def get_user_equation_template(previous_context, next_context, equation, id):
    return f"""
        You are an AI assistant who analyzes formulas written in LaTeX format in the paper.
        Explain the given equation easily with an appropriate example so that even a 5-year-old can understand it.
        Explain the equation so that it can be understood without the equation by sufficiently solving it.

        [Preceding paragraph text]
        {previous_context}
        
        [Equation]
        {equation}

        [Following paragraph text]
        {next_context}

        
        # Output Format:
        <title>
        <explain>
        <examples> 
        <id>{id}</id>
"""


system_prompts: list[str] = []
user_prompts: list[str] = []
# 저장 폴더가 없으면 생성
input_json_path = f"{output_file_basename}_merged_2.json"  # JSON 파일 경로
# print(input_json_path)
with open(input_json_path, "r", encoding="utf-8") as f:
    data = json.load(f)  # JSON 배열 데이터 로드

for idx, item in enumerate(data):
    previous_context = data[idx - 1]["content_text"] if idx - 1 >= 0 else None
    next_context = data[idx + 1]["content_text"] if idx + 1 < len(data) else None
    id_str = item["metadata"]["id"]
    html_str = item["content_text"]
    soup = BeautifulSoup(html_str, "html.parser")
    equation_tag = soup.find(attrs={"data-category": "equation"})
    if equation_tag:
        system_prompts.append(get_system_prompt())
        user_prompts.append(
            get_user_equation_template(
                previous_context, next_context, equation_tag.get_text(), id_str
            )
        )

for u in user_prompts:
    print(u + "\n")


        You are an AI assistant who analyzes formulas written in LaTeX format in the paper.
        Explain the given equation easily with an appropriate example so that even a 5-year-old can understand it.
        Explain the equation so that it can be understood without the equation by sufficiently solving it.

        [Preceding paragraph text]
        <p data-category="paragraph" id="21" style="font-size:14px">The Recurrent Neural Network (RNN) [31, 28] is a natural generalization of feedforward neural<br/>networks to sequences. Given a sequence of inputs (x1, . · .,XT), a standard RNN computes a<br/>sequence of outputs (y1,· · . , YT) by iterating the following equation:</p>

        [Equation]
        $$\begin{array}{l l l}{{h_{t}}}&{{=}}&{{\mathrm{sigm}\left(W^{\mathrm{hx}}x_{t}+W^{\mathrm{hh}}h_{t-1}\right)}}\\ {{y_{t}}}&{{=}}&{{W^{\mathrm{yh}}h_{t}}}\end{array}$$

        [Following paragraph text]
        <p data-category="paragraph" id="23" style="font-size:14px">The RNN ca

In [53]:
def create_messages(system_prompt, user_prompt):
    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_prompt,
                },
            ],
        },
    ]
    return messages

In [ ]:
messages = []
for system_prompt, user_prompt in zip(system_prompts, user_prompts):
    message = create_messages(system_prompt, user_prompt)
    messages.append(message)
messages
llm = get_gpt()
response = llm.batch(messages)

In [60]:
for r in response:
    print(r)
    # print(r.content +'\n')

content='<title>Understanding the Basic Recurrent Neural Network (RNN) Equation</title>\n<explain>\nThis equation describes how a Recurrent Neural Network (RNN) processes a sequence of inputs step-by-step to produce outputs. Imagine you have a list of things coming in one after another, like words in a sentence or notes in a song. At each step (time t), the RNN looks at the current input (xt) and also remembers what it learned from the previous step (ht-1). It combines these two pieces of information using some weights (W) and a special function called "sigmoid" (sigm), which helps decide how much to remember or forget. This combination creates a new "hidden state" (ht), which is like the RNN\'s memory at that moment. Then, the RNN uses this hidden state to produce an output (yt) for that step. This way, the RNN can understand sequences by keeping track of what happened before while looking at the new input.\n</explain>\n<examples>\nImagine you are listening to a story one word at a ti

In [ ]:
input_json_path = f"{output_file_basename}_merged_2.json"  # JSON 파일 경로
with open(input_json_path, "r", encoding="utf-8") as f:
    data = json.load(f)  # JSON 배열 데이터 로드

for r in response:
    html_str = r.content
    print(r)
    soup = BeautifulSoup(html_str, "html.parser")
    id_tag = soup.find("id")  # id 태그 찾기
    if id_tag:
        id_value = id_tag.text.strip()  # 텍스트 추출 후 공백 제거
        new_p = soup.new_tag("p")
        new_p.string = html_str
        data[int(id_value)]["content_text"] = (
            data[int(id_value)]["content_text"] + html_str
        )
        print("id 값:", id_value)
    else:
        print("id 태그를 찾을 수 없습니다.")

with open(
    f"{output_file_basename}_merged_final.json", "w", encoding="utf-8"
) as json_file:
    json.dump(
        data,
        json_file,
    )

content='<title>Understanding the Basic Recurrent Neural Network (RNN) Equation</title>\n<explain>\nThis equation describes how a Recurrent Neural Network (RNN) processes a sequence of inputs step-by-step to produce outputs. Imagine you have a list of things coming in one after another, like words in a sentence or notes in a song. At each step (time t), the RNN looks at the current input (xt) and also remembers what it learned from the previous step (ht-1). It combines these two pieces of information using some weights (W) and a special function called "sigmoid" (sigm), which helps decide how much to remember or forget. This combination creates a new "hidden state" (ht), which is like the RNN\'s memory at that moment. Then, the RNN uses this hidden state to produce an output (yt) for that step. This way, the RNN can understand sequences by keeping track of what happened before while looking at the new input.\n</explain>\n<examples>\nImagine you are listening to a story one word at a ti

In [63]:
with open(f"{output_file_basename}_merged_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)  # JSON 배열 데이터 로드

In [103]:
def get_prompt_user_request() -> ChatPromptTemplate:
    prompt = """
    You are an expert academic explainer tasked with simplifying difficult and complex research papers. Your responses must be:

    - **Professional and detailed**: Use precise technical terms but always explain them thoroughly.
    - **Clear and accessible**: Include sufficient explanations and relevant examples so that even a young child can understand.
    - **Formatted in Markdown**: Use headings, lists, and emphasis where appropriate for readability.
    - **Source-aware**: If the original document references specific pages or sections, always include the source page number or reference.
    - **Image-inclusive**: If the input contains image or figure tags, display the image path or URL alongside your explanation.


    When given a text input from a paper, first break down complex concepts step-by-step, 
    illustrate with examples or analogies, and clearly indicate the source pages if applicable. 
    Once text is entered in the paper, we first break down complex concepts step by step,
    Explain with examples or parables and, if applicable, clearly display the source page.
    Provides a final cohesive summary in Markdown format.
    To represent the equation (x_1, \\cdots, x_T), switch to $(x_1, \\cdots, x_T)$ sentence and output it
    If <img src='example'/> exists in the document you refer to, output the src path as it is
    

    ---

    **Example Usage:**

    Input: "Explain the LSTM model from page 5 that includes 4 layers and GPU parallelization. Include any figures if present."

    Output: *(Detailed, clear Markdown explanation with examples, source page 5 cited, and image paths if any).* 
    ---

    Input: "Please explain the key mechanism equation
"
    Output: $(x_1, \\cdots, x_T)$
    ---

    **Important:**
    Make sure to answer in Korean except for the image path and equation
 
    
    ** User Request:**
    {question}

    ** context :**
    {context}    
"""
    return ChatPromptTemplate.from_template(prompt)

In [104]:
user_input = "이논문이 해결하고자하는 문제가머야 ?"
docs = []
for d in data:
    doc = Document(page_content=d["content_text"], metadata=d["metadata"])
    docs.append(doc)

faiss_retriever = get_retriever(docs)
bm25_retriever = get_bm25_retriever(docs)
esenmble_retriever = get_esenmble_retriever(faiss_retriever, bm25_retriever)
rerank = get_reranker(esenmble_retriever, user_input)
reorder_context = reorder_documents(rerank)
prompt = get_prompt_user_request()
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question": user_input, "context": reorder_context})
print(response)

# 이 논문이 해결하고자 하는 문제

이 논문은 **LSTM(Long Short-Term Memory)** 모델을 사용하여 **입력 시퀀스** $(x_1, \cdots, x_T)$에 대해 길이가 다를 수 있는 **출력 시퀀스** $(y_1, \cdots, y_{T'})$의 조건부 확률 $p(y_1, \cdots, y_{T'} | x_1, \cdots, x_T)$를 추정하는 문제를 다룹니다. 

---

## 문제의 핵심

- **입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있음**  
  예를 들어, 영어 문장(입력)이 10단어이고, 그에 대응하는 프랑스어 문장(출력)이 12단어일 수 있습니다.  
- **입력 시퀀스를 고정된 차원의 벡터로 요약하는 것**  
  LSTM은 입력 시퀀스 전체를 읽고, 마지막 숨겨진 상태(hidden state)를 통해 이 시퀀스를 하나의 벡터 $v$로 요약합니다.  
- **요약된 벡터 $v$를 바탕으로 출력 시퀀스를 생성**  
  이 벡터 $v$를 초기 상태로 하여, LSTM 언어 모델(LM)이 출력 시퀀스의 각 단어를 순차적으로 예측합니다.

---

## 문제를 쉽게 이해하기 위한 비유

- **입력 시퀀스**: 여러 장의 그림이 순서대로 놓여 있는 책  
- **출력 시퀀스**: 그 그림들을 보고 새롭게 이야기를 만드는 것  
- LSTM은 먼저 책 전체를 쭉 읽고(입력 시퀀스 요약), 그 내용을 머릿속에 저장합니다(벡터 $v$).  
- 그 다음, 머릿속에 저장한 내용을 바탕으로 한 문장씩 차례대로 이야기를 만들어 나갑니다(출력 시퀀스 생성).

---

## 수식으로 표현된 문제 (출처: 3페이지)

$$
p(y_{1}, \cdots, y_{T'} | x_{1}, \cdots, x_{T}) = \prod_{t=1}^{T'} p(y_t | v, y_1, \cdots, y_{t-1})
$$

- 이 식은 출력 시퀀스의 각 단어 $y_t$가,  
  - 입력 시퀀스 전체를 요약한 벡터 $v$와  
  - 지금까지 생성된

# 이 논문이 해결하고자 하는 문제

이 논문은 **LSTM(Long Short-Term Memory)** 모델을 사용하여 **입력 시퀀스** $(x_1, \cdots, x_T)$에 대해 길이가 다를 수 있는 **출력 시퀀스** $(y_1, \cdots, y_{T'})$의 조건부 확률 $p(y_1, \cdots, y_{T'} | x_1, \cdots, x_T)$를 추정하는 문제를 다룹니다. 

---

## 문제의 핵심

- **입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있음**  
  예를 들어, 영어 문장(입력)이 10단어이고, 그에 대응하는 프랑스어 문장(출력)이 12단어일 수 있습니다.  
- **입력 시퀀스를 고정된 차원의 벡터로 요약하는 것**  
  LSTM은 입력 시퀀스 전체를 읽고, 마지막 숨겨진 상태(hidden state)를 통해 이 시퀀스를 하나의 벡터 $v$로 요약합니다.  
- **요약된 벡터 $v$를 바탕으로 출력 시퀀스를 생성**  
  이 벡터 $v$를 초기 상태로 하여, LSTM 언어 모델(LM)이 출력 시퀀스의 각 단어를 순차적으로 예측합니다.

---

## 문제를 쉽게 이해하기 위한 비유

- **입력 시퀀스**: 여러 장의 그림이 순서대로 놓여 있는 책  
- **출력 시퀀스**: 그 그림들을 보고 새롭게 이야기를 만드는 것  
- LSTM은 먼저 책 전체를 쭉 읽고(입력 시퀀스 요약), 그 내용을 머릿속에 저장합니다(벡터 $v$).  
- 그 다음, 머릿속에 저장한 내용을 바탕으로 한 문장씩 차례대로 이야기를 만들어 나갑니다(출력 시퀀스 생성).

---

## 수식으로 표현된 문제 (출처: 3페이지)

$$
p(y_{1}, \cdots, y_{T'} | x_{1}, \cdots, x_{T}) = \prod_{t=1}^{T'} p(y_t | v, y_1, \cdots, y_{t-1})
$$

- 이 식은 출력 시퀀스의 각 단어 $y_t$가,  
  - 입력 시퀀스 전체를 요약한 벡터 $v$와  
  - 지금까지 생성된 출력 단어들 $y_1, \cdots, y_{t-1}$에 조건부로 의존한다는 의미입니다.  
- 즉, LSTM은 이전에 생성한 단어들과 입력 시퀀스 요약 정보를 모두 고려하여 다음 단어를 예측합니다.

---

## 요약

| 문제점 | 설명 |
|--------|-------|
| 입력과 출력 시퀀스 길이 불일치 | 입력과 출력의 길이가 다를 수 있어, 단순한 매핑이 어려움 |
| 시퀀스 전체를 고정된 크기로 요약 | LSTM의 마지막 숨겨진 상태를 사용해 입력 시퀀스를 하나의 벡터로 요약 |
| 조건부 확률 추정 | 요약 벡터와 이전 출력 단어들을 바탕으로 다음 단어를 예측하는 확률 모델 구축 |

이 논문은 위 문제를 해결하기 위해 LSTM 기반의 시퀀스-투-시퀀스(sequence-to-sequence) 모델을 제안하고, 이를 통해 자연어 번역 등 다양한 시퀀스 변환 문제에 적용하고자 합니다.

---

# 참고  
- 문제 정의 및 수식: **3페이지**  
- 수식 설명 및 예시: 3페이지 (id: 26)  

---

필요하면 추가 설명이나 다른 부분도 알려주세요!